In [4]:
import tushare as ts
import pandas as pd

ts.set_token('530f6ccd413d494d4b798d9bfac281378ba24cf1abf875a8c14a3cb5')
global pro
pro = ts.pro_api()

In [45]:
#查询当前所有正常上市交易的股票列表
global stock_basic_info
stock_basic_info = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
stock_basic_info['exchange'] = stock_basic_info['ts_code'].apply(lambda x: x.split('.')[-1])
stock_basic_info.head()

,ts_code,symbol,name,area,industry,list_date,exchange
0,000001.SZ,000001,平安银行,深圳,银行,19910403,SZ
1,000002.SZ,000002,万科A,深圳,全国地产,19910129,SZ
2,000004.SZ,000004,国农科技,深圳,生物制药,19910114,SZ
3,000005.SZ,000005,世纪星源,深圳,环境保护,19901210,SZ
4,000006.SZ,000006,深振业A,深圳,区域地产,19920427,SZ


In [46]:
daily_data = pro.daily(trade_date='20191227')
daily_data.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000951.SZ,20191227,22.52,22.88,22.12,22.19,22.59,-0.40,-1.7707,47617.91,106759.336
1,603256.SH,20191227,15.50,15.62,14.88,14.98,15.47,-0.49,-3.1674,66859.58,102243.654
2,603259.SH,20191227,92.80,93.49,90.63,91.22,93.15,-1.93,-2.0719,96017.92,881148.712
3,000961.SZ,20191227,10.36,10.65,10.23,10.46,10.35,0.11,1.0628,305722.63,319773.194
4,000965.SZ,20191227,3.19,3.20,3.17,3.17,3.19,-0.02,-0.6270,37006.81,11790.405


In [47]:
# daily data for all stocks
daily_data = daily_data.merge(stock_basic_info.drop('symbol', axis=1), on='ts_code', how='left')
daily_data.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,name,area,industry,list_date,exchange
0,000951.SZ,20191227,22.52,22.88,22.12,22.19,22.59,-0.40,-1.7707,47617.91,106759.336,中国重汽,山东,汽车整车,19991125,SZ
1,603256.SH,20191227,15.50,15.62,14.88,14.98,15.47,-0.49,-3.1674,66859.58,102243.654,宏和科技,上海,玻璃,20190719,SH
2,603259.SH,20191227,92.80,93.49,90.63,91.22,93.15,-1.93,-2.0719,96017.92,881148.712,药明康德,江苏,化学制药,20180508,SH
3,000961.SZ,20191227,10.36,10.65,10.23,10.46,10.35,0.11,1.0628,305722.63,319773.194,中南建设,江苏,建筑工程,20000301,SZ
4,000965.SZ,20191227,3.19,3.20,3.17,3.17,3.19,-0.02,-0.6270,37006.81,11790.405,天保基建,天津,区域地产,20000406,SZ


In [82]:
# stocks whose price doubled in a month
def get_rocket_stock(start_date, end_date, min_increase_times, max_increase_times):
    '''
    get stock that get multiple times price increased
    '''
    start_state = pro.daily(trade_date=start_date)[['ts_code', 'close', 'vol', 'amount']]
    start_state.columns = ['ts_code', 'start_close', 'start_vol', 'start_amount']
    end_state = pro.daily(trade_date=end_date)
    end_state = end_state.merge(start_state, on='ts_code', how='left')
    end_state['increase_times'] = end_state['close'] / end_state['start_close']
    chosen_data = end_state[(end_state['increase_times'] > min_increase_times) & (end_state['increase_times'] <= max_increase_times)]
    chosen_data = chosen_data.merge(stock_basic_info, on='ts_code', how='left')
    return chosen_data

In [92]:
used_cols = ['ts_code', 'trade_date', 'open', 'high', 'low', 'close','change', 'pct_chg', 'vol', 'amount',
'start_close', 'increase_times', 'name', 'area', 'industry', 'exchange', 'list_date']

In [120]:
doubled_stock = get_rocket_stock('20190102', '20191202', 3, 10)[used_cols]

In [121]:
cond1 = doubled_stock['ts_code'].apply(lambda x:x[:3]) != '300'
cond2 = doubled_stock['close'] < 35
cond3 = doubled_stock['close'] > 6

In [123]:
doubled_stock[cond1 & cond2 & cond3]

,ts_code,trade_date,open,high,low,close,change,pct_chg,vol,amount,start_close,increase_times,name,area,industry,exchange,list_date
1,002552.SZ,20191202,25.43,26.29,24.70,25.74,-0.11,-0.4255,282000.87,720785.360,5.38,4.784387,宝鼎科技,浙江,机械基件,SZ,20110225
2,002201.SZ,20191202,22.32,23.38,22.00,23.10,0.76,3.4020,240177.97,547824.231,6.21,3.719807,九鼎新材,江苏,玻璃,SZ,20071226
4,002351.SZ,20191202,15.46,16.84,15.31,15.85,0.49,3.1901,682573.18,1097319.646,4.90,3.234694,漫步者,深圳,IT设备,SZ,20100205
6,002600.SZ,20191202,10.43,10.63,10.34,10.56,0.09,0.8596,794547.12,834518.531,2.48,4.258065,领益智造,广东,元器件,SZ,20110715
11,002100.SZ,20191202,12.69,13.10,12.60,12.91,0.69,5.6465,546474.62,703149.541,4.24,3.044811,天康生物,新疆,饲料,SZ,20061226
12,002463.SZ,20191202,22.49,23.05,22.32,22.64,0.15,0.6670,718757.08,1632311.365,7.19,3.148818,沪电股份,江苏,元器件,SZ,20100818
13,002123.SZ,20191202,22.19,23.07,21.80,22.44,0.30,1.3550,183598.13,415086.129,7.18,3.125348,梦网集团,深圳,互联网,SZ,20070328
18,000066.SZ,20191202,14.98,15.16,14.87,14.98,0.13,0.8754,492551.56,738430.831,4.71,3.180467,中国长城,深圳,IT设备,SZ,19970626
